In [71]:
import PyPDF2
import os
import string
import json
import re
from difflib import SequenceMatcher


### Article Cleaning

Note: the sample cleanedArticles and rawArticles is only the first few article folders & the first 10 valid articles within those folders (I didnt wanna clog the repo but wanted to show the output)


In [72]:
# Check to see if the word "abstract" appears before the word "introduction" in an article
def isValidOrder(text: str):
    abstractFound = False
    for i in range(len(text)):
        if not abstractFound and 'abstract' in text[0:i]:
            abstractFound = True
        if 'introduction' in text[0:i]:
            if abstractFound:
                return True
            return False
        # references showing up before "abstract" and "introduction"
        if 'references' in text[0:i]:
            return False

In [73]:

rootdir = 'articles'
validArticles = {}
for subdir, dirs, files in os.walk(rootdir):
    subdir = subdir.replace('\\', '')
    subdir = subdir.replace(f'{rootdir}', '')
    print('reading folder: ' + subdir)
    counter = 0
    for file in files:
        # im only taking the first 10 valid articles for each folder here so I can show input and output on github
        if counter == 9:
            break
        with open(f'{rootdir}/{subdir}/{file}', 'r', encoding='cp1252', errors='ignore') as f:
            text = " ".join(f.readlines())
            # only look for articles with "abstract", "introduction", and "references" to make cleaning possible
            if 'abstract' in text.lower() and 'introduction' in text.lower() and 'references' in text.lower():
                # need to make sure the show up in the right order
                if isValidOrder(text.lower()):
                    validArticles[file] = text 
                    counter += 1


reading folder: 
reading folder: 0704
reading folder: 0705
reading folder: 0706


In [74]:
import string
def cleanArticle(text: str):
    cleanedText = ""
    startingIndex = text.lower().find('introduction') + len('introduction')
    endingIndex = text.lower().rfind('references')
    # only read from the article's introduction section to its references section 
    text = text[startingIndex:endingIndex]
    # replace weird symbols with what they're supposed to be mapped to
    text = re.sub('-', '', text)
    text = re.sub('', 'ffi', text)
    text = re.sub('', 'fi', text)
    # remove emails
    text = re.sub('\S*@\S*\s?', '', text)
    # remove symbols
    text = re.sub("^(.*)$", "", text)
    if "Elba" in text:
        print(text)
        print(1)
    text = re.sub("^\d+\s|\s\d+\s|\s\d+$", " ", text)
    #     The \( and \) say we want to target the actual parenthesis.
    #     Then the parenthesis around the expression (.*?) say that we want to group what is inside
    #     Finally the .*? means that we want any character . and any repetition of that character *?.
    text = re.sub(r"\([^()]*\)", "", text)
    # remove extra spaces
    text = re.sub(' +', ' ', text)
    text = "".join(filter(lambda x: x in string.printable, text))
    text = re.sub(" ", "", text)
    text = " ".join(wordninja.split(text))
    text = re.sub(' \.', '.',text)
    return text

In [75]:
import os
import wordninja
articleIdentifiers = set()
for articleName in validArticles:
    if articleName[0:4] not in articleIdentifiers and not os.path.exists(f'cleanedArticles/{articleName[0:4]}') and not os.path.exists(f'rawArticles/{articleName[0:4]}'):
        articleIdentifiers.add(articleName[0:4])
        os.mkdir(f'cleanedArticles/{articleName[0:4]}')
        os.mkdir(f'rawArticles/{articleName[0:4]}')

    # writing raw articles to files (only doing this here because im taking the first 10 valid articles)
    text = validArticles[articleName]
    text_file = open(f"rawArticles/{articleName[0:4]}/{articleName}", "w")
    text_file.write(text)
    text_file.close()
    
    # writing cleaned articles to files
    cleanedText = cleanArticle(text)  
    # using word ninja to split words that were combined by the pdfR eader
    cleaned_text_file = open(f"cleanedArticles/{articleName[0:4]}/{articleName}", "w", encoding='utf-8')
    cleaned_text_file.write(cleanedText)
    cleaned_text_file.close()


arXiv:0704.0017v1  [astro-ph]  31 Mar 2007Mon. Not.R. Astron. Soc. 000, 000â€“000 (0000) Printed 11 February 2013 (MNL ATEXstyle ï¬lev2.2)
  SpectroscopicObservationsoftheIntermediatePolarEXHy draein
  Quiescence
  N. Mhlahlo1,2â‹†, D.A.H. Buckley2,V.S. Dhillon3, S.B. Potter2, B.Warner1and
  P.A. Woudt1
  1AstronomyDepartment, University ofCape Town, Rondebosch 7700, Cape Town, South Africa
  2South African Astronomical Observatory, Observatory 7935 , Cape Town, South Africa
  3Physics and Astronomy Department, University ofShefï¬eld, Shefï¬eld, S3 7RH,UK
  11 February 2013
  ABSTRACT
  ResultsfromspectroscopicobservationsoftheIntermedia tePolar(IP)EXHyainquiescence
  during1991and2001arepresented.Spin-modulatedradialv elocitiesconsistentwithanouter
  disc origin were detected for the ï¬rst time in an IP. The spin p ulsation was modulated with
  velocities near âˆ¼500âˆ’600 km sâˆ’1. These velocities are consistent with those of material
  circulating at the outer edge of the accretion